# Proyecto automatico: Planea y Estima

Hoy vamos a ver un sistema de agentes que basado en una descripcion y requerimientos de un proyecto, organiza los miembros de un equipo para realizar dicho proyecto

### Importamos dependencias


In [1]:
# # Warning control
# import warnings
# warnings.filterwarnings('ignore')

# Load environment variables
from dotenv import load_dotenv

load_dotenv()

import os
import yaml
from crewai import Agent, Task, Crew

In [2]:
os.environ['OPENAI_MODEL_NAME'] = 'gpt-4o-mini'

### Cargamos las instrucciones

In [3]:
# Define file paths for YAML configurations
files = {
    'agents': 'config/agents.yaml',
    'tasks': 'config/tasks.yaml'
}

# Load configurations from YAML files
configs = {}
for config_type, file_path in files.items():
    with open(file_path, 'r') as file:
        configs[config_type] = yaml.safe_load(file)

# Assign loaded configurations to specific variables
agents_config = configs['agents']
tasks_config = configs['tasks']

## Structured Output con Pydantic

Pydantic es una biblioteca de Python que se utiliza para validar y manejar datos de manera sencilla y eficiente. Se basa en **tipado estático** y utiliza anotaciones de tipo para definir estructuras de datos, **garantizando que los datos recibidos cumplan con el formato esperado**.

**Validación automática:** Verifica que los datos cumplan con los tipos definidos en los modelos. 

**Conversión de datos:** Transforma los datos al tipo correcto si es posible (ejemplo: convertir "123" en int(123)).

**Modelos basados en clases:** Se definen usando clases de Python con anotaciones de tipo.

**Mensajes de error claros:** En caso de errores de validación, Pydantic proporciona mensajes detallados.

**Compatibilidad con FastAPI:** Es ampliamente utilizado en FastAPI para manejar y validar datos de peticiones HTTP.

**Soporte para estructuras anidadas:** Permite crear modelos con estructuras complejas.


El equivalente en Typescript es **zod**

## Importamos Pydantic

Creamos la estructura de la Tarea, tendra los campos de :

- Nombre

- Horas estimadas

- Recursos necesarios para cumplir con la tarea

In [4]:
from typing import List
from pydantic import BaseModel, Field

class TaskEstimate(BaseModel):
    task_name: str = Field(..., description="Name of the task")
    estimated_time_hours: float = Field(..., description="Estimated time to complete the task in hours")
    required_resources: List[str] = Field(..., description="List of resources required to complete the task")



## La estructura del Milestone tendra:

- Nombre del milestone

- Tareas relacionadas con ese milestone

In [5]:
# Milestone = Hito, Un paso en el proceso para cumplir el objetivo
class Milestone(BaseModel):
    milestone_name: str = Field(..., description="Name of the milestone")
    tasks: List[str] = Field(..., description="List of task IDs associated with this milestone")

## Plan de proyecto

- Lista de tareas

- Lista de Milestones

In [6]:
class ProjectPlan(BaseModel):
    tasks: List[TaskEstimate] = Field(..., description="List of tasks with their estimates")
    milestones: List[Milestone] = Field(..., description="List of project milestones")

### Inicializamos los agentes y las tareas.

Se va a ejecutar ed forma secuencial

In [7]:
# Creating Agents
project_planning_agent = Agent(
  config=agents_config['project_planning_agent']
)

estimation_agent = Agent(
  config=agents_config['estimation_agent']
)

resource_allocation_agent = Agent(
  config=agents_config['resource_allocation_agent']
)

# Creating Tasks
task_breakdown = Task(
  config=tasks_config['task_breakdown'],
  agent=project_planning_agent
)

time_resource_estimation = Task(
  config=tasks_config['time_resource_estimation'],
  agent=estimation_agent
)

resource_allocation = Task(
  config=tasks_config['resource_allocation'],
  agent=resource_allocation_agent,
  output_pydantic=ProjectPlan # This is the structured output we want
)

# Creating Crew
crew = Crew(
  agents=[
    project_planning_agent,
    estimation_agent,
    resource_allocation_agent
  ],
  tasks=[
    task_breakdown,
    time_resource_estimation,
    resource_allocation
  ],
  verbose=True
)

### Creamos el input para la crew

In [8]:
from IPython.display import display, Markdown

project = 'chatbot'
industry = 'Technology'
project_objectives = 'Create a chatbot for customer support'
team_members = """
- Saul (Software Manager)
- Alejandro (AI Engineer)
- Mauro (Designer)
"""
project_requirements = """
The chatbot must be able to answer frequently asked questions about products and services.
It should allow escalation to a human agent when necessary.
Ability to interpret natural language queries and respond accurately.
Support for multiple languages (at least English and Spanish).
Integration with databases to retrieve up-to-date information about customers and products.
Ability to handle multiple conversations simultaneously.
"""

# Format the dictionary as Markdown for a better display in Jupyter Lab
formatted_output = f"""
**Project Type:** {project}

**Project Objectives:** {project_objectives}

**Industry:** {industry}

**Team Members:**
{team_members}
**Project Requirements:**
{project_requirements}
"""
# Display the formatted output as Markdown
display(Markdown(formatted_output))


**Project Type:** chatbot

**Project Objectives:** Create a chatbot for customer support

**Industry:** Technology

**Team Members:**

- Saul (Software Manager)
- Alejandro (AI Engineer)
- Mauro (Designer)

**Project Requirements:**

The chatbot must be able to answer frequently asked questions about products and services.
It should allow escalation to a human agent when necessary.
Ability to interpret natural language queries and respond accurately.
Support for multiple languages (at least English and Spanish).
Integration with databases to retrieve up-to-date information about customers and products.
Ability to handle multiple conversations simultaneously.



## Ejecutamos la Crew

In [9]:
# The given Python dictionary
inputs = {
  'project_type': project,
  'project_objectives': project_objectives,
  'industry': industry,
  'team_members': team_members,
  'project_requirements': project_requirements
}

# Run the crew
result = crew.kickoff(
  inputs=inputs
)

# Agent: The Ultimate Project Planner
## Task: Carefully analyze the project_requirements for the chatbot project and break them down into individual tasks. Define each task's scope in detail, set achievable timelines, and ensure that all dependencies are accounted for:

The chatbot must be able to answer frequently asked questions about products and services.
It should allow escalation to a human agent when necessary.
Ability to interpret natural language queries and respond accurately.
Support for multiple languages (at least English and Spanish).
Integration with databases to retrieve up-to-date information about customers and products.
Ability to handle multiple conversations simultaneously.


Team members:

- Saul (Software Manager)
- Alejandro (AI Engineer)
- Mauro (Designer)




# Agent: The Ultimate Project Planner
## Final Answer: 
### Comprehensive Breakdown of Chatbot Project Tasks

**Project Scope Overview:**
The project aims to develop a customer support chatbot that can e

In [10]:
import pandas as pd

costs = 0.150 * (crew.usage_metrics.prompt_tokens + crew.usage_metrics.completion_tokens) / 1_000_000
print(f"Total costs: ${costs:.4f}")

# Convert UsageMetrics instance to a DataFrame
df_usage_metrics = pd.DataFrame([crew.usage_metrics.dict()])
df_usage_metrics

Total costs: $0.0012


,total_tokens,prompt_tokens,cached_prompt_tokens,completion_tokens,successful_requests
0,8111,4899,0,3212,3


Haber utilizado la validacion con pydantic, nos facilita mucho a la hora de manejar los datos obtenidos.

Convertimos en diccionario el output del modelo

In [11]:
result.pydantic.dict()

{'tasks': [{'task_name': 'Requirement Gathering and Analysis',
   'estimated_time_hours': 40.0,
   'required_resources': ['Stakeholders', 'Documentation Software'],
   'responsible_member': 'Saul'},
  {'task_name': 'Design Chatbot Flows',
   'estimated_time_hours': 80.0,
   'required_resources': ['Design Prototyping Tools', 'User Testing Tools'],
   'responsible_member': 'Mauro'},
  {'task_name': 'Natural Language Processing Development',
   'estimated_time_hours': 120.0,
   'required_resources': ['NLP Frameworks', 'Training Datasets'],
   'responsible_member': 'Alejandro'},
  {'task_name': 'Database Integration',
   'estimated_time_hours': 80.0,
   'required_resources': ['Database Management Systems',
    'API Development Tools'],
   'responsible_member': 'Alejandro'},
  {'task_name': 'User Interface Design',
   'estimated_time_hours': 120.0,
   'required_resources': ['Prototyping Software', 'Graphic Design Resources'],
   'responsible_member': 'Mauro'},
  {'task_name': 'Development o

Convertimos el diccionario en tabla html y podemos visualizarlo de forma sencilla y limpia.

Por un lado las tareas y por otro los mailstones

In [12]:
tasks = result.pydantic.dict()['tasks']
df_tasks = pd.DataFrame(tasks)

# Display the DataFrame as an HTML table
df_tasks.style.set_table_attributes('border="1"').set_caption("Task Details").set_table_styles(
    [{'selector': 'th, td', 'props': [('font-size', '120%')]}]
)

,task_name,estimated_time_hours,required_resources,responsible_member
0,Requirement Gathering and Analysis,40.000000,"['Stakeholders', 'Documentation Software']",Saul
1,Design Chatbot Flows,80.000000,"['Design Prototyping Tools', 'User Testing Tools']",Mauro
2,Natural Language Processing Development,120.000000,"['NLP Frameworks', 'Training Datasets']",Alejandro
3,Database Integration,80.000000,"['Database Management Systems', 'API Development Tools']",Alejandro
4,User Interface Design,120.000000,"['Prototyping Software', 'Graphic Design Resources']",Mauro
5,Development of Chatbot Architecture,160.000000,"['Cloud Hosting Services', 'API Management Tools']",Saul
6,Testing Phase,80.000000,"['Testing Frameworks', 'Bug Tracking System']",Alejandro
7,Launch Preparation,80.000000,"['Documentation Tools', 'Marketing Platforms']",Saul
8,Post-Launch Support and Updates,40.000000,"['Feedback Collection Tools', 'Analytics Software']",Mauro


In [13]:
milestones = result.pydantic.dict()['milestones']
df_milestones = pd.DataFrame(milestones)

# Display the DataFrame as an HTML table
df_milestones.style.set_table_attributes('border="1"').set_caption("Task Details").set_table_styles(
    [{'selector': 'th, td', 'props': [('font-size', '120%')]}]
)

,milestone_name,tasks
0,Completion of Requirement Gathering,['Requirement Gathering and Analysis']
1,Design Phase Completion,"['Design Chatbot Flows', 'User Interface Design']"
2,NLP Development Completion,"['Natural Language Processing Development', 'Database Integration']"
3,Development Completion,['Development of Chatbot Architecture']
4,Testing Phase Completion,['Testing Phase']
5,Launch Readiness,['Launch Preparation']
6,Ongoing Support and Updates,['Post-Launch Support and Updates']
